
# HealPIX Tutorial

This tutorial demonstrates key HealPIX functionality including:
1. Visualization using double pixelization
2. Zonal averaging
3. Regridding between different grids


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import earth2grid
from earth2grid import healpix

# Create a sample HealPIX grid
level = 4  # Resolution level (higher = finer resolution)
grid = healpix.Grid(level=level, pixel_order=healpix.XY())

# Create a sample field with lat/lon dependence
def create_sample_field(lat, lon):
    """Create a sample field with lat/lon dependence"""
    lat = torch.from_numpy(lat)
    lon = torch.from_numpy(lon)
    return torch.cos(torch.deg2rad(lat)) * torch.sin(2 * torch.deg2rad(lon))

# Generate the sample field
field = create_sample_field(grid.lat, grid.lon)

# 1. Visualization using double pixelization
Double pixelization provides a visually appealing way to view HealPIX data
without interpolation, preserving the native pixel structure



In [ ]:
# Convert to double pixelization
field_double = healpix.to_double_pixelization(field)

plt.figure(figsize=(12, 4))
plt.imshow(field_double)
plt.colorbar(label='Field Value')
plt.title('Double Pixelization Visualization')
plt.show()

# 2. Zonal Averaging
Compute the zonal average of the field (average along latitude circles)



In [ ]:
zonal_avg = healpix.zonal_average(field)

# Plot the zonal average
plt.figure(figsize=(8, 4))
plt.plot(grid.lat[::100], zonal_avg[::100])  # Plot every 100th point for clarity
plt.xlabel('Latitude (degrees)')
plt.ylabel('Zonal Average')
plt.title('Zonal Average of Sample Field')
plt.grid(True)
plt.show()

# 3. Regridding
Demonstrate regridding between different grid types



In [ ]:
# Create a lat-lon grid
nlat, nlon = 33, 64
latlon_grid = earth2grid.latlon.equiangular_lat_lon_grid(nlat, nlon)

# Create regridder from HealPIX to lat-lon
regridder = earth2grid.get_regridder(grid, latlon_grid)

# Regrid the field
field_regridded = regridder(field)

# Plot original and regridded fields
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Original HealPIX field
im1 = ax1.scatter(grid.lon, grid.lat, c=field, s=1)
ax1.set_title('Original HealPIX Field')
plt.colorbar(im1, ax=ax1)

# Regridded field
im2 = ax2.pcolormesh(latlon_grid.lon, latlon_grid.lat, field_regridded)
ax2.set_title('Regridded to Lat-Lon')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

# Additional: Regridding between HealPIX grids of different resolutions
Create a higher resolution HealPIX grid



In [ ]:
high_res_grid = healpix.Grid(level=level+1, pixel_order=healpix.XY())

# Create regridder
high_res_regridder = earth2grid.get_regridder(grid, high_res_grid)

# Regrid the field
field_high_res = high_res_regridder(field)

# Plot original and high-resolution fields
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Original resolution
im1 = ax1.scatter(grid.lon, grid.lat, c=field, s=1)
ax1.set_title(f'Original HealPIX (Level {level})')
plt.colorbar(im1, ax=ax1)

# High resolution
im2 = ax2.scatter(high_res_grid.lon, high_res_grid.lat, c=field_high_res, s=0.5)
ax2.set_title(f'High Resolution HealPIX (Level {level+1})')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()